In [53]:
import pandas as pd
import geopandas as gpd
import numpy as np
import contextily as ctx
import matplotlib
import matplotlib.pyplot as plt
from shapely import wkt
from pathlib import Path
import os

In [54]:
img_dir = Path("../img_natl")
if not os.path.isdir("../img_natl"):
    os.makedirs("../img_natl")

In [60]:
df = pd.read_csv("../natl_mep_matrix.csv")
df['geometry'] = df.geometry.apply(wkt.loads)
df = gpd.GeoDataFrame(df, crs="EPSG:4326")

In [56]:
df

,geometry,hex_id,lat,lon,mep,mep_entertainment,mep_food,mep_healthcare,mep_jobs,mep_retail,mep_services,mode,population
0,"POLYGON ((-170.81076 -14.35791, -170.78572 -14...",869a14077ffffff,-14.325730,-170.819622,3189.846436,3173.116396,9.803002,5.042827,1.042691,0.841519,0.000000,walk,0.000000
1,"POLYGON ((-170.69284 -14.32222, -170.66781 -14...",869a1401fffffff,-14.290042,-170.701740,7709.981414,7669.544265,23.694233,12.188708,2.520224,2.033985,0.000000,walk,0.000000
2,"POLYGON ((-170.75180 -14.34007, -170.72677 -14...",869a1400fffffff,-14.307896,-170.760684,3363.579718,3345.938486,10.336918,5.317482,1.099480,0.887352,0.000000,walk,0.000000
3,"POLYGON ((-170.75180 -14.34007, -170.72677 -14...",869a1400fffffff,-14.307896,-170.760684,14682.736684,14605.728974,45.122831,23.211934,4.799464,3.873480,0.000000,bike,0.000000
4,"POLYGON ((-170.63387 -14.30435, -170.60885 -14...",869a140afffffff,-14.272169,-170.642789,193.111605,192.098777,0.593469,0.305290,0.063124,0.050945,0.000000,bike,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
743963,"POLYGON ((-99.80821 41.27680, -99.76997 41.294...",86261e34fffffff,41.310343,-99.811486,295.614230,35.368321,15.625852,185.178979,2.718835,2.361994,54.360249,walk,32.627187
743964,"POLYGON ((-99.80821 41.27680, -99.76997 41.294...",86261e34fffffff,41.310343,-99.811486,2072.522069,337.799674,132.979470,1267.130473,17.924841,16.044768,300.642843,bike,32.627187
743965,"POLYGON ((-99.80821 41.27680, -99.76997 41.294...",86261e34fffffff,41.310343,-99.811486,23616.602819,5130.406043,1779.151155,13798.035519,192.411891,177.113496,2539.484715,drive,32.627187
743966,"POLYGON ((-99.27648 41.13650, -99.23828 41.154...",862618c4fffffff,41.170075,-99.279566,52.333121,0.000000,0.000000,7.827056,1.998540,0.841073,41.666453,walk,45.650729


In [35]:
names = df.name.unique()
for name in names:
    img_filename = name.replace(' ', '_').replace("/", "_") + ".png"
    if not os.path.exists(img_filename):
        fig, ax = plt.subplots()
        natl_df = df[(df['mode']=='drive')&(df['name']==name)].copy()
        natl_df.to_crs("EPSG:3857").plot("mep", ax=ax, alpha=.5, cmap='Oranges', edgecolor="k", linewidth=0.1, legend=True)
        mep_w = np.average(natl_df.mep, weights=natl_df.population)
        ax.set_title(f"{name} MEP: {mep_w}")
        ax.set_xticks([])
        ax.set_yticks([])
        ctx.add_basemap(ax=ax, source=ctx.providers.USGS.USTopo)
        fig.savefig(img_dir / img_filename, dpi=300, bbox_inches="tight")
        plt.close()

In [62]:

# national
outs = [
    ("national", (-180, -64)),
    ("continental", (-124.76, -66.95))
]
providers = [ctx.providers.USGS.USImagery, ctx.providers.USGS.USTopo]
for name, (minx, maxx) in outs:
    for mode in ["linear", "log"]:
        for provider in providers:
            img_filename = f"{name}-{mode}-{provider['name']}.png"
            if not (img_dir / img_filename).exists():
                fig, ax = plt.subplots()
                    
                natl_df = df[(df['mode']=='drive')&(df['lon']>minx)&(df['lon']<maxx)].copy()

                if mode == "linear":
                    natl_df.to_crs("EPSG:3857").plot("mep", ax=ax, alpha=.66, cmap='viridis', linewidth=0.0, legend=True)
                elif mode == "log":
                    norm=matplotlib.colors.LogNorm(vmin=natl_df.mep.min(), vmax=natl_df.mep.max())
                    natl_df.to_crs("EPSG:3857").plot("mep", ax=ax, alpha=.66, cmap='viridis', norm=norm, linewidth=0.0, legend=True)
                else:
                    raise "bad mode"

                mep_w = np.average(natl_df.mep, weights=natl_df.population)
                ax.set_title(f"{name} MEP: {mep_w}")
                ax.set_xticks([])
                ax.set_yticks([])
                ctx.add_basemap(ax=ax, source=provider)
                fig.savefig(img_dir / img_filename, dpi=600, bbox_inches="tight")
                plt.close()